## Tutorial: BEST Test

In [ ]:
#| hide
#skip
! [ -e /content ] && pip install -Uqq pyndamics3 emcee # upgrade pyndamics3 on colab

In [ ]:
%matplotlib inline
from pylab import *

In [ ]:
from sie.mcmc import StatsModel

In [ ]:
drug = (101,100,102,104,102,97,105,105,98,101,100,123,105,103,100,95,102,106,
        109,102,82,102,100,102,102,101,102,102,103,103,97,97,103,101,97,104,
        96,103,124,101,101,100,101,101,104,100,101)
placebo = (99,101,100,101,102,100,97,101,104,101,102,102,100,105,88,101,100,
           104,100,100,100,101,102,103,97,101,101,100,101,99,101,100,100,
           101,100,99,101,100,102,99,100,99)

In [ ]:
model=StatsModel()
model.add_data(drug=drug,placebo=placebo)

pooled=np.concatenate([drug,placebo])
S=mean(pooled)
M=std(pooled)                      
model.extra(S=S,M=M,mn=0.001*S,mx=1000*S)


model.add("μ_drug ~ Normal(M,1000*S)")
model.add("μ_placebo ~ Normal(M,1000*S)")
model.add("σ_drug ~ Uniform(mn,mx)")
model.add("σ_placebo ~ Uniform(mn,mx)")
model.add("ν ~ Exponential(29,offset=1)")
model.add("drug ~ StudentT(ν,μ_drug,σ_drug)")
model.add("placebo ~ StudentT(ν,μ_placebo,σ_placebo)")


model.initialize()
model


Parameters
----------
    {'μ_drug': μ_drug, 'μ_placebo': μ_placebo, 'σ_drug': σ_drug, 'σ_placebo': σ_placebo, 'ν': ν}
Extra
-----
    ['S', 'M', 'mn', 'mx']
Data
----
    ['drug', 'placebo']
Prior
-----
    ['μ_drug ~ Normal(M,1000*S)', 'μ_placebo ~ Normal(M,1000*S)', 'σ_drug ~ Uniform(mn,mx)', 'σ_placebo ~ Uniform(mn,mx)', 'ν ~ Exponential(29,offset=1)']
Likelihood
----------
    []
Data Parameters
---------------
    ['drug ~ StudentT(ν,μ_drug,σ_drug)', 'placebo ~ StudentT(ν,μ_placebo,σ_placebo)']
        

In [ ]:
print(model.make_func())

def _lnprior(θ,slices,extra={}):
    S=extra['S']
    M=extra['M']
    mn=extra['mn']
    mx=extra['mx']
    μ_drug=θ[slices.μ_drug]
    μ_placebo=θ[slices.μ_placebo]
    σ_drug=θ[slices.σ_drug]
    σ_placebo=θ[slices.σ_placebo]
    ν=θ[slices.ν]

    _value=0

    _value+=Normal(M,1000*S)(μ_drug)
    _value+=Normal(M,1000*S)(μ_placebo)
    _value+=Uniform(mn,mx)(σ_drug)
    _value+=Uniform(mn,mx)(σ_placebo)
    _value+=Exponential(29,offset=1)(ν)

    return _value


def _init_prior(nwalkers,ndim,data,slices,extra={}):
    S=extra['S']
    M=extra['M']
    mn=extra['mn']
    mx=extra['mx']
    drug=data['drug']
    placebo=data['placebo']

    _pos=np.zeros((nwalkers,ndim))
    μ_drug=_pos[:,slices.μ_drug]=init_Normal(M,1000*S)(nwalkers)
    μ_placebo=_pos[:,slices.μ_placebo]=init_Normal(M,1000*S)(nwalkers)
    σ_drug=_pos[:,slices.σ_drug]=init_Uniform(mn,mx)(nwalkers)
    σ_placebo=_pos[:,slices.σ_placebo]=init_Uniform(mn,mx)(nwalkers)
    ν=_pos[:,slices.ν]=init_Exponential(29,offse

In [ ]:
model.run_mcmc(800,repeat=3)
model.plot_chains()

Sampling Prior...
Done.
0.41 s
Running MCMC 1/3...
emcee: Exception while calling your likelihood function:
  params: [-1.76373583e+10  4.90159284e+10 -5.81801780e+16 -1.58753585e+18
 -6.26093289e+13]
  args: []
  kwargs: {}
  exception:


/Users/bblais/Documents/Git/sie/sie/mcmc.py:228: RuntimeWarning: invalid value encountered in log
  values=N*(gammaln((df+1)/2.0)-0.5*log(df*np.pi)-gammaln(df/2.0)-np.log(sd))+(-(df+1)/2.0)*np.log(1+t**2/df)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/work/lib/python3.11/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/bblais/Documents/Git/sie/sie/mcmc.py", line 613, in __call__
    return self._lnposterior(θ)
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/bblais/Documents/Git/sie/sie/mcmc.py", line 608, in _lnposterior
    _value+=self._lnlikelihood(θ,self.data,self.slices,self.extra_params)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 58, in _lnlikelihood
  File "/Users/bblais/Documents/Git/sie/sie/mcmc.py", line 231, in _StudentT
    raise ValueError('NaN in StudentT',df,mu,sd)
ValueError: ('NaN 

ValueError: ('NaN in StudentT', array([-6.26093289e+13]), array([-1.76373583e+10]), array([-5.8180178e+16]))

In [ ]:
print(model.make_func())